In [87]:
import numpy as np
import pandas as pd

## 1. Import all the data files

In [16]:
order_report=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Company X - Order Report.xlsx')
pin_code_zone=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Company X - Pincode Zones.xlsx')
SKU_master=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Company X - SKU Master.xlsx')
Courier_Company_Invoice=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Courier Company - Invoice.xlsx')
Courier_Company_Rates=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Courier Company - Rates.xlsx')
Output_sample_summary=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Expected_Result.xlsx','Summary')
Output_sample_calculation=pd.read_excel(r'F:\Cointab Data Analyst - Challenge\Expected_Result.xlsx','Calculations')


In [48]:
Result2=pd.DataFrame(columns=Output_sample_calculation.columns)
Result2.columns

Index(['Order ID', 'AWB Number', 'Total weight as per X (KG)',
       'Weight slab as per X (KG)', 'Total weight as per Courier Company (KG)',
       'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
       'Delivery Zone charged by Courier Company',
       'Expected Charge as per X (Rs.)',
       'Charges Billed by Courier Company (Rs.) ',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

## 2. Generating Total weight as per company X
group by order ID: for combining with different dataset

In [7]:
Company_X=order_report.merge(SKU_master,on='SKU')
Company_X['Total weight as per X (KG)']=Company_X['Order Qty']*Company_X['Weight (g)']/1000
Company_X=Company_X.groupby(by=['ExternOrderNo']).sum()
Company_X.reset_index(inplace=True)
Company_X.rename({'ExternOrderNo':'Order ID'},axis=1,inplace=True)
Company_X

,Order ID,Order Qty,Weight (g),Total weight as per X (KG)
0,2001806210,2.0,220,0.220
1,2001806226,2.0,240,0.480
2,2001806229,3.0,500,0.500
3,2001806232,8.0,377,1.302
4,2001806233,2.0,245,0.245
...,...,...,...,...
119,2001821995,2.0,477,0.477
120,2001822466,10.0,352,1.376
121,2001823564,6.0,336,0.672
122,2001825261,13.0,611,1.557


## 3. assigning Order ID to pin code zone file

In [21]:
dfa = pin_code_zone.drop_duplicates(subset=['Customer Pincode'])
dfb=Courier_Company_Invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pin_code=dfb.merge(dfa,on='Customer Pincode')
pin_code.rename({'Zone':'Delivery Zone as per X'},axis=1,inplace=True)
pin_code

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone as per X
0,2001806232,507101,Forward charges,121003,d
1,2001806273,486886,Forward charges,121003,d
2,2001806408,532484,Forward charges,121003,d
3,2001806458,143001,Forward charges,121003,b
4,2001807012,515591,Forward charges,121003,d
...,...,...,...,...,...
119,2001811039,302020,Forward charges,121003,b
120,2001811305,302020,Forward charges,121003,b
121,2001812941,325207,Forward charges,121003,b
122,2001809383,303702,Forward and RTO charges,121003,b


## 4.preparation of company_X data

In [28]:
Company_X=Company_X.merge(pin_code,on='Order ID')
Company_X

,Order ID,Order Qty,Weight (g),Total weight as per X (KG),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone as per X
0,2001806210,2.0,220,0.220,140604,Forward charges,121003,b
1,2001806226,2.0,240,0.480,723146,Forward charges,121003,d
2,2001806229,3.0,500,0.500,421204,Forward charges,121003,d
3,2001806232,8.0,377,1.302,507101,Forward charges,121003,d
4,2001806233,2.0,245,0.245,263139,Forward charges,121003,b
...,...,...,...,...,...,...,...,...
119,2001821995,2.0,477,0.477,342008,Forward charges,121003,b
120,2001822466,10.0,352,1.376,342301,Forward charges,121003,b
121,2001823564,6.0,336,0.672,492001,Forward and RTO charges,121003,d
122,2001825261,13.0,611,1.557,517128,Forward and RTO charges,121003,d


## 5. Function for assigning weight slab

In [29]:
def weight_slab(weight):
    i=round(weight%1,1)
    if i==0.0:
        return weight
    elif i>0.5:
        return int(weight)+1.0
    else:
        return int(weight)+0.5
  

## 6. generation of Weight slab column

In [32]:
Company_X['weight slab as per X (KG)']=(Company_X['Total weight as per X (KG)']).apply(weight_slab)
Courier_Company_Invoice.rename({'Charged Weight':'Total weight as per Courier Company (KG)'},axis=1,inplace=True)
Courier_Company_Invoice['Weight slab charged by Courier Company (KG)']=(Courier_Company_Invoice['Total weight as per Courier Company (KG)']).apply(weight_slab)

## 7. Computation of Expected charge

In [88]:
total_Expected_charge=[]
for i in range(len(Company_X)):
    fwd_category='fwd_'+Company_X['Delivery Zone as per X'][i]
    fwd_fixed=Courier_Company_Rates[fwd_category+'_fixed'][0]
    fwd_additional=Courier_Company_Rates[fwd_category+'_additional'][0]
    rto_category='rto_'+Company_X['Delivery Zone as per X'][i]
    rto_fixed=Courier_Company_Rates[rto_category+'_fixed'][0]
    rto_additional=Courier_Company_Rates[rto_category+'_additional'][0]
    if Company_X['Type of Shipment'][i]=='Forward charges':
        if Company_X['weight slab as per X (KG)'][i]<=0.5:
            total_Expected_charge.append(fwd_fixed)
        else:
            additional_weight=(Company_X['weight slab as per X (KG)'][i]-0.5)/0.5
            total_Expected_charge.append(fwd_fixed+additional_weight*fwd_additional)
    if Company_X['Type of Shipment'][i]=='Forward and RTO charges':
        if Company_X['weight slab as per X (KG)'][i]<=0.5:
            total_Expected_charge.append(fwd_fixed+rto_fixed)
        else:
            additional_weight=(Company_X['weight slab as per X (KG)'][i]-0.5)/0.5
            total_Expected_charge.append(fwd_fixed+additional_weight*(fwd_additional+rto_additional))

Company_X['Expected Charge as per X (Rs.)']=total_Expected_charge


In [89]:
Company_X.head(5)

,Order ID,Order Qty,Weight (g),Total weight as per X (KG),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone as per X,weight slab as per X (KG),Expected Charge as per X (Rs.)
0,2001806210,2.0,220,0.220,140604,Forward charges,121003,b,0.5,33.0
1,2001806226,2.0,240,0.480,723146,Forward charges,121003,d,0.5,45.4
2,2001806229,3.0,500,0.500,421204,Forward charges,121003,d,0.5,45.4
3,2001806232,8.0,377,1.302,507101,Forward charges,121003,d,1.5,135.0
4,2001806233,2.0,245,0.245,263139,Forward charges,121003,b,0.5,33.0


## 8. Data merging to get desired Output_file

In [39]:
Courier_Company_Invoice.columns

Index(['AWB Code', 'Order ID', 'Total weight as per Courier Company (KG)',
       'Warehouse Pincode', 'Customer Pincode', 'Zone', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Weight slab charged by Courier Company (KG)'],
      dtype='object')

In [40]:
Subset_Courier_Company=Courier_Company_Invoice[['AWB Code', 'Order ID', 'Total weight as per Courier Company (KG)',
        'Zone','Billing Amount (Rs.)', 'Weight slab charged by Courier Company (KG)']]

In [90]:
merged_data=Company_X.merge(Subset_Courier_Company,on='Order ID',how='left')
merged_data.head(5)

,Order ID,Order Qty,Weight (g),Total weight as per X (KG),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone as per X,weight slab as per X (KG),Expected Charge as per X (Rs.),AWB Code,Total weight as per Courier Company (KG),Zone,Billing Amount (Rs.),Weight slab charged by Courier Company (KG)
0,2001806210,2.0,220,0.220,140604,Forward charges,121003,b,0.5,33.0,1091117221940,2.92,b,174.5,3.0
1,2001806226,2.0,240,0.480,723146,Forward charges,121003,d,0.5,45.4,1091117222065,0.68,d,90.2,1.0
2,2001806229,3.0,500,0.500,421204,Forward charges,121003,d,0.5,45.4,1091117222080,0.71,d,90.2,1.0
3,2001806232,8.0,377,1.302,507101,Forward charges,121003,d,1.5,135.0,1091117222124,1.30,d,135.0,1.5
4,2001806233,2.0,245,0.245,263139,Forward charges,121003,b,0.5,33.0,1091117222135,0.78,b,61.3,1.0


In [56]:
Result2['Order ID']=merged_data['Order ID']
Result2['AWB Number']=merged_data['AWB Code']
Result2['Total weight as per X (KG)']=merged_data['Total weight as per X (KG)']
Result2['Weight slab as per X (KG)']=merged_data['weight slab as per X (KG)']
Result2['Total weight as per Courier Company (KG)']=merged_data['Total weight as per Courier Company (KG)']
Result2['Weight slab charged by Courier Company (KG)']=merged_data['Weight slab charged by Courier Company (KG)']
Result2['Delivery Zone as per X']=merged_data['Delivery Zone as per X']
Result2['Delivery Zone charged by Courier Company']=merged_data['Zone']
Result2['Expected Charge as per X (Rs.)']=merged_data['Expected Charge as per X (Rs.)']
Result2['Charges Billed by Courier Company (Rs.) ']=merged_data['Billing Amount (Rs.)']
Result2['Difference Between Expected Charges and Billed Charges (Rs.)']=Result2['Expected Charge as per X (Rs.)']-Result2['Charges Billed by Courier Company (Rs.) ']


In [91]:
Result2.head(5)

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.0,61.3,-28.3


## 9. Generate OutPut1 (summary) 

In [77]:

correctly_charged=Result2[Result2['Difference Between Expected Charges and Billed Charges (Rs.)']==0.0]
correctly=["Total Orders where X has been correctly charged",len(correctly_charged),sum(correctly_charged['Charges Billed by Courier Company (Rs.) '])]

over_charged=Result2[Result2['Difference Between Expected Charges and Billed Charges (Rs.)']<0.0]
over=["Total Orders where X has been over charged",len(over_charged),np.absolute(sum(over_charged['Difference Between Expected Charges and Billed Charges (Rs.)']))]

under_charged=Result2[Result2['Difference Between Expected Charges and Billed Charges (Rs.)']>0.0]
under=["Total Orders where X has been under charged",len(under_charged),np.absolute(sum(under_charged['Difference Between Expected Charges and Billed Charges (Rs.)']))]

Result1=pd.DataFrame([correctly,over,under],columns=['Description','Count','Amount (Rs.)'])


In [78]:
Result1

,Description,Count,Amount (Rs.)
0,Total Orders where X has been correctly charged,18,1493.2000
1,Total Orders where X has been over charged,96,4931.3656
2,Total Orders where X has been under charged,10,381.9504


## 10. Save the output to the excel sheet

In [86]:
writer = pd.ExcelWriter('F:\\Cointab_test_Rahul_Harod_IIT_Bombay.xlsx', engine='xlsxwriter')
Result1.to_excel(writer, sheet_name='Summary')
Result2.to_excel(writer,sheet_name='Calculations')
writer.save()